In [94]:
import cv2 as cv
import numpy as np
import scipy
import pickle
import random
import os
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.cluster import KMeans,MiniBatchKMeans

In [95]:
directory='/Users/maddisiddart/Desktop/lessdataset'
images=os.listdir(directory)

In [96]:
#finding descriptors
desc=[]
key_list=[]
X=[]
sift = cv.xfeatures2d.SIFT_create()
for img_name in images:
    img_path=os.path.join(directory,img_name)
    img=cv.imread(img_path)
    '''plt.figure()
    plt.imshow(img)
    plt.show() '''
    desc.append([])
    key_list.append([])
    kps=sift.detect(img)
    kp1, des1 = sift.compute(img, kps)
    desc[-1].append(des1)
    key_list[-1].append(kp1)
    
    for d in des1:
        X.append(d)

In [97]:
#clustering of key points to make dictionary using k means
dictionary_size=50
kmeans = KMeans(n_clusters=dictionary_size, init='k-means++', max_iter=300, n_init=10, random_state=0).fit(X)
centers=kmeans.cluster_centers_
print("clustered")





clustered


In [98]:
#flann matcher to match key points to words in dictionary
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50) 

flann = cv.FlannBasedMatcher(index_params,search_params)
arr=[]
for i in range(len(desc)):
    matches = flann.knnMatch(np.asarray(desc[i][0],np.float32),np.asarray(centers,np.float32),k=2)
    #for j in range(len(matches)):
        #arr[-1].append(matches[j][0].trainIdx)
    arr.append([matches[j][0].trainIdx for j in range(len(matches))])


In [99]:
#creating a histogram
doc=[]
for i in range(len(arr)): 
    hist=[0]*(dictionary_size)
    for j in range(0,dictionary_size):
        l = len((np.where(np.array(arr[i]) == j))[0])
        hist[j] = l
    doc.append(hist)
print("histogram with frequency of words in an image done (method 1)")

histogram with frequency of words in an image done


In [100]:
doc_word=[]
for i in range(len(arr)):
    hist=[0]*dictionary_size
    c={}
    for j in range(len(arr[i])):
        if (arr[i][j] in c):
            c[arr[i][j]]+=1
        else:
            c[arr[i][j]]=1
    for k in c:
        hist[k]=c[k]
    doc_word.append(hist)
print("histogram with frequency of words in an image done (method 2)")       
    

In [106]:
print(doc_word)

[[15, 0, 12, 14, 15, 6, 7, 0, 6, 6, 7, 4, 8, 1, 4, 4, 14, 12, 10, 10, 10, 6, 6, 1, 4, 17, 30, 11, 7, 5, 9, 11, 9, 18, 17, 17, 12, 3, 9, 13, 7, 7, 10, 5, 13, 11, 6, 9, 6, 4], [0, 0, 4, 7, 6, 2, 0, 2, 11, 0, 0, 1, 6, 3, 0, 1, 2, 1, 0, 0, 1, 0, 2, 0, 1, 1, 1, 1, 6, 2, 2, 0, 0, 6, 2, 0, 0, 3, 1, 7, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0], [1, 0, 1, 2, 1, 0, 3, 1, 1, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 1, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 2, 3, 0, 0, 0, 0, 0, 0], [1, 0, 1, 1, 0, 3, 2, 1, 12, 3, 1, 1, 2, 3, 3, 0, 2, 1, 3, 0, 1, 0, 2, 0, 2, 2, 0, 8, 1, 11, 0, 3, 0, 5, 2, 0, 1, 1, 7, 3, 3, 0, 1, 0, 1, 1, 2, 5, 3, 0], [2, 0, 0, 0, 1, 1, 3, 0, 0, 0, 0, 2, 1, 0, 1, 0, 1, 3, 0, 0, 0, 3, 0, 0, 1, 1, 0, 0, 5, 0, 4, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0], [2, 0, 0, 3, 1, 0, 2, 0, 0, 2, 1, 1, 0, 0, 4, 1, 2, 2, 1, 0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 6, 0], [12, 0, 3, 12, 5, 2, 13, 0, 4, 2, 5, 7, 0, 1, 11, 2, 15, 9, 6, 